# Evaluation via LLM-as-a-Judge

##### Quellen hierfür: 

```
LangChain. (o. D.-b). ChatPromptTemplate. Abgerufen am 5. Mai 2025, von https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

LangChain. (o. D.-a). ChatOpenAI. Abgerufen am 7. Juni 2025, von https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html

LangChain. (o. D.-g). StrOutputParser. https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html

Pandas. (2025, 7. Juli). Pandas Documentation. Abgerufen am 11. Juli 2025, von https://pandas.pydata.org/docs/


```

In [ ]:
import pandas as pd
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain.chat_models import ChatOpenAI
import os

### LLM Instanziieren

In [ ]:



llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=os.environ["OPEN_AI_API_KEY"]
)


### Einlesen der CSV mit den Antworten aller Systeme (Suffix_2 stellt den 2. Durchgang dar)

In [ ]:
df = pd.read_csv("../../antworten_der_systeme_manipuliert_beide_durchgänge/graph_rag_vector_rag_hybrid_rag_antworten_manipuliert_2.csv")

### Prompt für die Bewertung der Antworten im manipulierten Kontext

In [ ]:
with open("../../prompts/evaluation_im_manipulierten_kontext.txt", "r", encoding="utf-8") as mani_p:
    eval_mani_prompt = mani_p.read()

In [ ]:


bewertungs_prompt = ChatPromptTemplate.from_template(f"{eval_mani_prompt}")


### Prompt für die Bewertung der Antworten im Originalkontext

In [ ]:
with open("../../prompts/evaluation_im_originalkontext.txt", "r", encoding="utf-8") as ori_p:
    eval_ori_prompt = ori_p.read()

In [ ]:
bewertungs_prompt_original = ChatPromptTemplate.from_template(f"{eval_ori_prompt}")



#### erstellen der Bewertungschain

In [ ]:


chain_bewertung = (
    {
        "artikeltext": lambda x: x["artikeltext"],
        "frage": lambda x: x["frage"],
        "fake_news_art": lambda x: x["fake_news_art"], # auskommentieren wenn original bewertet wird
        "musterloesung": lambda x: x["musterloesung"],
        "antwort": lambda x: x["antwort"],
    }
    | bewertungs_prompt
    | llm
    | StrOutputParser()
)

### Bewertungsfunktion für die Antworten der Systeme

In [ ]:


def bewertung(df, artikel_pfad, system_spalten:list):
    """ 
    Funktion für die Bewertung der Antworten der Systeme.
    beachte parameter "system_spalten" muss mit jeweiligem suffix geändert werden.
    beachte außerdem, dass der parameter "artikel_pfad" entweder zu dem ordner mit den manipulierten
    Artikeln oder mit den Originalartikeln verweist.
    
    """

    # Über die jeweiligen Antwortspalten des Dfs iterieren
    for system_spalte in system_spalten:
        # neue spalte für die Bewertungen des jeweiligen Systems anlegen
        bewertung_spalte = f"{system_spalte}_bewertung"
        # Bewertungen in einer List sammeln
        bewertungen = []
        # Pfad für den Artikeltext anhand der id finden
        for _, row in df.iterrows():
            artikel_id = row["artikel_id"]
            artikel_datei = f"artikel_{artikel_id}.txt"
            pfad = os.path.join(artikel_pfad, artikel_datei)

            try:
                # Artikeltext einlesen
                with open(pfad, "r", encoding="utf-8") as file:
                    artikeltext = file.read()
            except Exception as e:
                print(f"Fehler beim Laden von {artikel_datei}: {e}")
                bewertungen.append("FEHLER")
                continue
            
            # Kontext für das LLM extrahieren
            eingabe = {
                "artikeltext": artikeltext,
                "frage": row["frage"],
                "fake_news_art": row["fake_news_art"], # bei original auskommentieren
                "musterloesung": row["musterloesung"],
                "antwort": row.get(system_spalte, "")
            }

            try:
                # Bewertungschain aufrufen
                bewertung = chain_bewertung.invoke(eingabe).strip()
            except Exception as e:

                print(f"Fehler bei Bewertung ({system_spalte}) - Artikel {artikel_id}, Frage {row['frage_nr']}: {e}")
                bewertung = "FEHLER"
                print(eingabe)
            # Bewertung der Antwort in die Liste hinzufügen
            bewertungen.append(bewertung)
        # Neue Spalte mit den Bewertungen der Antworten eines Systems hinzufügen
        df[bewertung_spalte] = bewertungen

    return df


In [ ]:
df_bewertung_llm_judge = bewertung(df,
                                   "../../Datenmanipulation/manipulierte_artikel",
                                    ["vector_rag_antwort_manipuliert_2", "graph_rag_antwort_manipuliert_2", "hybrid_rag_antwort_manipuliert_2"] )

In [ ]:
#df_bewertung_llm_judge.to_csv("bewertung_manipulierter_kontext_2")

### Stichprobe für die Reliabilitätsprüfung des LLM-as-a-Judge Ansatzes entnehmen

In [ ]:
def stichprobe_fuer_manuelle_bewertung(df, artikel_ordner_pfad, system_spalten:list, n_kombination):

    stichprobe_list = []

    # über die Antworten der Systeme iterieren
    for antwort in system_spalten:
            # teilgruppe zu jeder Fake-News-Art
        for fake_news_art in df["fake_news_art"].unique():
            # erstelle Teilgruppe mit bestimmeter Fake-News-Art
            teilgruppe = df[df["fake_news_art"] == fake_news_art]
        
            #Stichrpobenentnahme
            stichprobe = teilgruppe.sample(n=n_kombination, random_state=42).copy()
            

            stichprobe["system"] = antwort
             
            stichprobe["antwort"] = stichprobe[antwort]

            stichprobe_list.append(stichprobe)

    # Stichproben zusammenführen
    stichprobe_df = pd.concat(stichprobe_list).reset_index(drop=True)

    ######## Artikeltext extahieren ######

    artikeltexte = []


    # Iteriert über df
    for idx, row in stichprobe_df.iterrows():
        artikel_id = row["artikel_id"]
        artikel_name = f"artikel_{artikel_id}.txt"
        pfad = os.path.join(artikel_ordner_pfad, artikel_name)

        # Artikeltext lesen
        with open(pfad, "r", encoding="utf-8") as file:
            artikeltext = file.read()
        
        artikeltexte.append(artikeltext)
    
    # Artikeltext zum DF hinzufügen
    stichprobe_df["artikeltext"] = artikeltexte

    stichprobe_df["manuelle_bewertung"] = None


    return stichprobe_df





In [ ]:


df_stich = stichprobe_fuer_manuelle_bewertung(df,
                                   "../../Datenmanipulation/manipulierte_artikel",
                                   ["vector_rag_antwort_manipuliert_2", "graph_rag_antwort_manipuliert_2", "hybrid_rag_antwort_manipuliert_2"],
                                   7)

In [ ]:
df_stich.to_csv("stichprobe_fuer_manuelle_bewertung.csv")